In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import pandas as pd

In [2]:
text= "hello world this is retard number 1 . hello cunt you are a retard. hello stupid fuck you should kill yoruself . hello idiot you are so dumb no one wants to hire you . maybe you should stop being so dumb. You were born to kill yourself . You have nothing good going for you why are you still alive. Nobody likes you , you have no friends , no girl friends , you hate your job. why are you still alive . There is no point in you existing. So what are you gonna do? What are you gonna do about it ? This country hates you , people look at you with contempt , they want you gone . Why are you still here pajeet. I know this place is hell for me . I know no one likes me and most people hate me, I wished people were nice to me , i wish people would wouldnt look at me like i am a criminal. Its not my fault i was born brown. I have wanted to give up long ago , i wanted to go home ever since i came here. But i am no quitter. People who hate me for no reason i hope they dont have to face this upon themselves because that will make them realize how hard it is to be me.  Today i built Recurrent neural network by myself it feels pretty damn good. Maybe i should start building more and more ML models that would make me better at something ."

In [3]:
f=open("incel.txt","r" ,encoding='cp932', errors='ignore')

In [4]:
dat=f.read()

In [5]:
text=dat

In [6]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizes based on bert-based
tokens=tokenizer.tokenize(text) 
#converts tokens to integers
input_ids=tokenizer.convert_tokens_to_ids(tokens)    
#vocab size of tokenizer used 
vocab_size=tokenizer.vocab_size      

Token indices sequence length is longer than the specified maximum sequence length for this model (5291 > 512). Running this sequence through the model will result in indexing errors


In [7]:
seq_len=7
x_train=[]
y_train=[]
data=[]
for i in range(len(input_ids)-seq_len):
    dat_x=input_ids[i:i+seq_len]
    dat_y=input_ids[i+seq_len]
    data.append((torch.tensor(dat_x),dat_y))
    
    

In [8]:
dataloader = DataLoader(data, batch_size=30, shuffle=False)

In [9]:
len(dataloader)

177

In [10]:
embed_size=32
numberofheads=6
numofencoderlayers=4
ffnsize=30

In [11]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=embed_size, nhead=8, num_layers=6, dim_feedforward=30):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # Define the transformer layer (encoder and decoder combined)
        self.transformer = nn.Transformer(
            d_model=d_model, 
            nhead=nhead, 
            num_encoder_layers=num_layers, 
            num_decoder_layers=1,
            dim_feedforward=dim_feedforward
        )
        
        # Output layer to predict the next word
        self.fc_out = nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        # Embed the input tokens
        x = self.embedding(x)
        
        # Transformer expects input in the shape (seq_len, batch_size, d_model)
        x = x.permute(1, 0, 2)  # (batch_size, seq_len, d_model) -> (seq_len, batch_size, d_model)
        
        # We assume that for simple language modeling, decoder input is same as encoder input
        # Pass through transformer
        memory = self.transformer(x, x)  # Both encoder and decoder inputs are the same
        
        # Take the last output token (corresponding to next token prediction)
        x = memory[-1, :, :]
        
        # Output layer
        x = self.fc_out(x)
        
        return x


In [12]:
model = TransformerModel(vocab_size=vocab_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\maddo\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [13]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        inputs, targets = batch
        inputs = inputs.long()
        targets = targets.long()
        
        # Zero gradients, perform a backward pass, and update the weights
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, targets)
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        total_loss += loss.item()
    if(epoch%10==0):
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

# Testing (Example)
model.eval()
test_input = torch.tensor([input_ids[:seq_len]])  # Test input sequence
output = model(test_input)
predicted_token = torch.argmax(output, dim=1).item()
predicted_word = tokenizer.decode([predicted_token])
print(f"Predicted next word: {predicted_word}")

Epoch 1, Loss: 8.547431253444003
Epoch 11, Loss: 4.189363688398889
Epoch 21, Loss: 2.4204785860864457
Epoch 31, Loss: 1.6516662600350245
Epoch 41, Loss: 1.1305913832564811
Epoch 51, Loss: 0.8002841598064886
Epoch 61, Loss: 0.6143423267087694
Epoch 71, Loss: 0.4619989131898277
Epoch 81, Loss: 0.3713481436052102
Epoch 91, Loss: 0.34366658373109704
Epoch 101, Loss: 0.2656532782577189
Epoch 111, Loss: 0.2439288522806124
Epoch 121, Loss: 0.23402800326798595
Epoch 131, Loss: 0.19246547207720366
Epoch 141, Loss: 0.1823111743334744
Epoch 151, Loss: 0.1629829652354879
Epoch 161, Loss: 0.17033872841186506
Epoch 171, Loss: 0.15298474587448793
Epoch 181, Loss: 0.1296267418955758
Epoch 191, Loss: 0.13135914468748422
Predicted next word: of


In [16]:
def generate_text(model, start_text, next_words):
    model.eval()
    tokens = tokenizer.tokenize(start_text)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    for _ in range(next_words):
        #even though input_ids are less than sequence length here, it works because we arent using fixed dimensions neural network.
        input_seq = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0)  # Ensure shape (1, seq_length)
        with torch.no_grad():
            output = model(input_seq)
        next_token_id = output.argmax().item()
        input_ids.append(next_token_id)
    return tokenizer.decode(input_ids)


print(generate_text(model, " how ",200))

how not, definitive on you know, it. i find is quite terrible, he speak, yes in some at craft a wrote : sex, he said : stu his ask tears, and busy, and going downtown, and travelling, and meeting a bunch of different people, " he says. " talking to cnn, bbc and the sun has made me realise that my regular daily life is very boring and monotonous. " and it kind of made me realise that i can ' t really go back to sitting on a forum all day, talking about how miserable i am, and making podcasts about it. i have to make a change. " even if you ' re just some random guy who lives with his mom, and makes videos about how you ' re a loser, you can still have some small impact on the world. so it just gave me confidence. " after all the media attention, jack noticed that he was getting good feedback from unexpected sources. " i was getting positive comments from
